<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# Bayesian Personalized Ranking (BPR)

This notebook serves as an introduction to Bayesian Personalized Ranking (BPR) model for implicit feedback.  In this tutorial, we focus on learning the BPR model using matrix factorization approach, hence, the model is sometimes also named BPRMF.

The implementation of the model is from [Cornac](https://github.com/PreferredAI/cornac), which is a framework for recommender systems with a focus on models leveraging auxiliary data (e.g., item descriptive text and image, social network, etc).

## 0 Global Settings and Imports

In [1]:
import os
import sys
import cornac
import pandas as pd

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k, mae, rmse, novelty, historical_item_novelty, user_item_serendipity, user_serendipity, serendipity, catalog_coverage, distributional_coverage
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

c:\Users\julia\anaconda3\envs\conda_python3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


System version: 3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]
Cornac version: 2.3.0


In [2]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# top k items to recommend
TOP_K = 10

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100

## 1 BPR Algorithm

### 1.1 Personalized Ranking from Implicit Feedback

The task of personalized ranking aims at providing each user a ranked list of items (recommendations). This is very common in scenarios where recommender systems are based on implicit user behavior (e.g. purchases, clicks). The available observations are only positive feedback where the non-observed ones are a mixture of real negative feedback and missing values.

One usual approach for item recommendation is directly predicting a preference score $\hat{x}_{u,i}$ given to item $i$ by user $u$. This method uses the **pointwise loss** to optimize the model. Methods based on pointwise learning usually follow a regression framework by minimizing the  squared loss between $\hat{x}_{u,i}$ its target value ${x}_{u,i}$. Unfortunately, this method doesn't work well with ranking.

<p align="center">
<img src="https://recodatasets.z20.web.core.windows.net/images/bpr1.png" width="400px">
</p>

In the implicit feedback scenario the observed data is positive feedback. As it is shown in the figure above, the observed data (image to the left) is known, but the unobserved data is unknown. The negative data is usually generated by filling the matrix with 0 (image to the right). 

Using the pointwise loss, the model is trained to predict as 1 the elements in the observed dataset and 0 for the rest. The problem with this approach is that all elements the model should rank in the future are presented as negative feedback during training but in reality, some of them are unknown items and some of them are true negatives. 

BPR uses a different approach by using item pairs $(i, j)$ and optimizing for the correct ranking given preference of user $u$, thus, there are notions of *positive* and *negative* items. The training data $D_S : U \times I \times I$ is defined as:

$$D_S = \{(u, i, j) \mid i \in I^{+}_{u} \wedge j \in I \setminus I^{+}_{u}\}$$

where user $u$ is assumed to prefer $i$ over $j$ (i.e. $i$ is a *positive item* and $j$ is a *negative item*).

This approach uses the **pairwise loss**. The intuition behind it is that observed entries should be ranked higher than the unobserved ones. Then instead of minimizing the loss between $\hat{x}_{u,i}$ and ${x}_{u,i}$, pairwise learning maximizes the margin between the observed entry $\hat{x}_{u,i}$ and the unobserved entry $\hat{x}_{u,j}$.

<p align="center">
<img src="https://recodatasets.z20.web.core.windows.net/images/bpr2.png" width="400px">
</p>

In the figure above, pairwise learning is explained. For each user, a pairwise matrix of items is generated. The plus indicates that the user prefers item *i* over item *j*, and the minus is that the user prefers item *j* over *i*. The interrogation indicates that we don't know the user's preference.

The pairwise setup has several advantages:
* The training set is defined as positive, negative and unknown preferences. 
* The missing values between two non-observed items are exactly the item pairs that have to be ranked by the model.
* The training data is created for the actual objective of ranking.
* Pairwise ranking is not restricted to implicit feedback and can be used in explicit feedback by setting user preferences as pairs (i.e. user *u* prefers item *a* more than item *b*).

### 1.2 Pairwise Objective Function

From the Bayesian perspective, BPR maximizes the posterior probability over the model parameters $\Theta$ by optimizing the likelihood function $p(i >_{u} j | \Theta)$ and the prior probability $p(\Theta)$.

$$p(\Theta \mid >_{u}) \propto p(i >_{u} j \mid \Theta) \times p(\Theta)$$

The joint probability of the likelihood over all users $u \in U$ can be simplified to:

$$ \prod_{u \in U} p(>_{u} \mid \Theta) = \prod_{(u, i, j) \in D_S} p(i >_{u} j \mid \Theta) $$

The individual probability that a user $u$ prefers item $i$ to item $j$ can be defined as:

$$ p(i >_{u} j \mid \Theta) = \sigma (\hat{x}_{uij}(\Theta)) $$

where $\sigma$ is the logistic sigmoid:

$$ \sigma(x) = \frac{1}{1 + e^{-x}} $$

The preference scoring function $\hat{x}_{uij}(\Theta)$ could be an arbitrary real-valued function of the model parameter $\Theta$.  Thus, it makes BPR a general framework for modeling the relationship between triplets $(u, i, j)$ where different model classes like matrix factorization could be used for estimating $\hat{x}_{uij}(\Theta)$.

For the prior, one of the common pratices is to choose $p(\Theta)$ following a normal distribution, which results in a nice form of L2 regularization in the final log-form of the objective function.

$$ p(\Theta) \sim N(0, \Sigma_{\Theta}) $$

To reduce the complexity of the model, all parameters $\Theta$ are assumed to be independent and having the same variance, which gives a simpler form of the co-variance matrix $\Sigma_{\Theta} = \lambda_{\Theta}I$.  Thus, there are less number of hyperparameters to be determined.

The final objective of the maximum posterior estimator:

$$ J = \sum_{(u, i, j) \in D_S} \text{ln } \sigma(\hat{x}_{uij}) - \lambda_{\Theta} ||\Theta||^2 $$

where $\lambda_\Theta$ are the model specific regularization parameters.


### 1.3 Learning with Matrix Factorization

#### Stochastic Gradient Descent

As the defined objective function is differentiable, gradient descent based method for optimization is naturally adopted.  The gradient of the objective $J$ with respect to the model parameters:

$$
\begin{align}
\frac{\partial J}{\partial \Theta} & = \sum_{(u, i, j) \in D_S} \frac{\partial}{\partial \Theta} \text{ln} \ \sigma(\hat{x}_{uij}) - \lambda_{\Theta} \frac{\partial}{\partial \Theta} ||\Theta||^2 \\
& \propto \sum_{(u, i, j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot  \frac{\partial}{\partial \Theta} \hat{x}_{uij} - \lambda_{\Theta} \Theta
\end{align}
$$

Due to slow convergence of full gradient descent, we prefer using stochastic gradient descent to optimize the BPR model.  For each triplet $(u, i, j) \in D_S$, the update rule for the parameters:

$$ \Theta \leftarrow \Theta + \alpha \Big( \frac{e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot \frac{\partial}{\partial \Theta} \hat{x}_{uij} + \lambda_\Theta \Theta \Big) $$

#### Matrix Factorization for Preference Approximation

As mentioned earlier, the preference scoring function $\hat{x}_{uij}(\Theta)$ could be approximated by any real-valued function.  First, the estimator $\hat{x}_{uij}$ is decomposed into:

$$ \hat{x}_{uij} = \hat{x}_{ui} - \hat{x}_{uj} $$

The problem of estimating $\hat{x}_{ui}$ is a standard collaborative filtering formulation, where matrix factorization approach has shown to be very effective.  The prediction formula can written as dot product between user feature vector $w_u$ and item feature vector $h_i$:

$$ \hat{x}_{ui} = \langle w_u , h_i \rangle = \sum_{f=1}^{k} w_{uf} \cdot h_{if} $$

The  derivatives of matrix factorization with respect to the model parameters are:

$$
\frac{\partial}{\partial \theta} \hat{x}_{uij} = 
\begin{cases}
    (h_{if} - h_{jf})  & \text{if } \theta = w_{uf} \\
    w_{uf}             & \text{if } \theta = h_{if} \\
    -w_{uf}            & \text{if } \theta = h_{jf} \\
    0                  & \text{else}
\end{cases}
$$

In theory, any kernel can be used to estimate $\hat{x}_{ui}$ besides the dot product $ \langle \cdot , \cdot \rangle $.  For example, k-Nearest-Neighbor (kNN) has also been shown to achieve good performance.

#### Analogies to AUC optimization

By optimizing the objective function of BPR model, we effectively maximizing [AUC](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) measurement. To keep the notebook focused, please refer to the [paper](https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf) for details of the analysis (Section 4.1.1).

## 2 Cornac implementation of BPR

BPR is implemented in the [Cornac](https://cornac.readthedocs.io/en/latest/index.html) framework as part of the model collections.
* Detailed documentations of the BPR model in Cornac can be found [here](https://cornac.readthedocs.io/en/latest/models.html#bayesian-personalized-ranking-bpr).
* Source codes of the BPR implementation is available on the Cornac Github repository, which can be found [here](https://github.com/PreferredAI/cornac/blob/master/cornac/models/bpr/recom_bpr.pyx).


## 3 Cornac BPR movie recommender


### 3.1 Load and split data

To evaluate the performance of item recommendation, we adopted the provided `python_random_split` tool for the consistency.  Data is randomly split into training and test sets with the ratio of 75/25.


Note that Cornac also cover different [built-in schemes](https://cornac.readthedocs.io/en/latest/eval_methods.html) for model evaluation.

In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

100%|██████████| 4.81k/4.81k [00:01<00:00, 3.67kKB/s]


,userID,itemID,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [4]:
train, test = python_random_split(data, 0.75)

### 3.2 Cornac Dataset

To work with models implemented in Cornac, we need to construct an object from [Dataset](https://cornac.readthedocs.io/en/latest/data.html#module-cornac.data.dataset) class.

Dataset Class in Cornac serves as the main object that the models will interact with.  In addition to data transformations, Dataset provides a bunch of useful iterators for looping through the data, as well as supporting different negative sampling techniques.

In [5]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 943
Number of items: 1642


### 3.3 Train the BPR model

The BPR has a few important parameters that we need to consider:

- `k`: controls the dimension of the latent space (i.e. the size of the vectors  $w_u$  and  $h_i$ ).
- `max_iter`: defines the number of iterations of the SGD procedure.
- `learning_rate`: controls the step size $\alpha$ in the gradient update rules.
- `lambda_reg`: controls the L2-Regularization $\lambda$ in the objective function.

Note that different values of `k` and `max_iter` will affect the training time.

We will here set `k` to 200, `max_iter` to 100, `learning_rate` to 0.01, and `lambda_reg` to 0.001. To train the model, we simply need to call the `fit()` method.

In [6]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [7]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 100/100 [00:06<00:00, 15.72it/s, correct=92.09%, skipped=9.09%]

Optimization finished!
Took 6.3869 seconds for training.


### 3.4 Prediction and Evaluation

Now that our model is trained, we can produce the ranked lists for recommendation.  Every recommender models in Cornac provide `rate()` and `rank()` methods for predicting item rated value as well as item ranked list for a given user.  To make use of the current evaluation schemes, we will through `predict()` and `predict_ranking()` functions inside `cornac_utils` to produce the predictions.

Note that BPR model is effectively designed for item ranking.  Hence, we only measure the performance using ranking metrics.

In [8]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 1.6415 seconds for prediction.


In [9]:
all_predictions.head()

,userID,itemID,prediction
0,1,1,3.917794
3,1,4,2.972839
5,1,6,-0.691013
7,1,8,2.865882
9,1,10,1.761243


In [10]:
all_predictions[all_predictions["userID"] == 1]

,userID,itemID,prediction
0,1,1,3.917794
3,1,4,2.972839
5,1,6,-0.691013
7,1,8,2.865882
9,1,10,1.761243
...,...,...,...
1637,1,1676,-2.454599
1638,1,1678,-2.861341
1639,1,1679,-2.831001
1640,1,1680,-2.897071


In [11]:
all_predictions[all_predictions["userID"] == 1].sort_values(by="prediction", ascending=False)


,userID,itemID,prediction
99,1,100,4.518073
180,1,181,4.115204
167,1,168,4.063992
203,1,204,4.052636
237,1,238,4.051291
...,...,...,...
1640,1,1680,-2.897071
1627,1,1663,-2.909554
1567,1,1588,-2.940504
1630,1,1666,-2.941876


In [12]:
rows, columns = test.shape
rows

25000

In [13]:
test[test["userID"] == 1].sort_values(by="rating", ascending=False)

,userID,itemID,rating
4280,1,82,5.0
78817,1,216,5.0
33909,1,197,5.0
36844,1,119,5.0
3431,1,221,5.0
...,...,...,...
31650,1,8,1.0
43438,1,110,1.0
62649,1,263,1.0
8745,1,259,1.0


In [14]:
test[(test["userID"] == 1) & (test["rating"] == 5)]

,userID,itemID,rating
17672,1,100,5.0
38193,1,181,5.0
5533,1,6,5.0
6541,1,258,5.0
3431,1,221,5.0
4280,1,82,5.0
33909,1,197,5.0
82786,1,204,5.0
32236,1,1,5.0
36844,1,119,5.0


In [15]:
merged_df = test.merge(all_predictions, on=["userID", "itemID"], how="inner")[["userID", "itemID", "rating", "prediction"]]
merged_df

,userID,itemID,rating,prediction
0,877,381,4.0,2.171268
1,815,602,3.0,2.331577
2,94,431,4.0,3.498765
3,416,875,2.0,0.481092
4,500,182,2.0,1.970066
...,...,...,...,...
24948,399,684,3.0,3.161546
24949,222,580,3.0,-0.502356
24950,551,162,5.0,0.939652
24951,803,988,1.0,3.471947


In [16]:
user_counts = test["userID"].value_counts().reset_index()
user_counts.columns = ["userID", "count"]
user_counts

,userID,count
0,405,188
1,655,169
2,13,161
3,450,139
4,276,123
...,...,...
937,808,2
938,547,1
939,812,1
940,400,1


In [17]:
max_predictions = all_predictions.loc[all_predictions.groupby("userID")["prediction"].idxmax(), ["userID", "itemID", "prediction"]]
max_predictions

,userID,itemID,prediction
99,1,100,4.518073
1941,2,300,5.528343
3569,3,286,6.200184
5211,4,286,5.099172
6728,5,161,5.088063
...,...,...,...
1540489,939,294,5.375546
1542095,940,258,4.153353
1543767,941,288,4.491458
1545407,942,286,4.787055


In [ ]:
k = 1

eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision_k = precision_at_k(test, all_predictions, col_prediction='prediction', k=10)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall_k = recall_at_k(test, all_predictions, col_prediction='prediction', k=10)
# eval_f1
eval_mae = mae(test, all_predictions, col_prediction='prediction')
eval_rmse = rmse(test, all_predictions, col_prediction='prediction')
# eval_mrr
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
# eval_intra_list_diversity
# eval_user_coverage
# eval_item_coverage
# eval_personalization

eval_novelty = novelty(train, max_predictions)
eval_historical_item_novelty = historical_item_novelty(train, max_predictions)
eval_user_item_serendipity = user_item_serendipity(train, max_predictions)
eval_user_serendipity = user_serendipity(train, max_predictions)
eval_serendipity = serendipity(train, max_predictions)
eval_catalog_coverage = catalog_coverage(train, max_predictions)
eval_distributional_coverage = distributional_coverage(train, max_predictions)

c:\Users\julia\anaconda3\envs\conda_python3.9\lib\site-packages\recommenders\evaluation\python_evaluation.py:1435: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  avg_novelty = reco_item_novelty.agg({"product": "sum"})[0] / n_recommendations
c:\Users\julia\anaconda3\envs\conda_python3.9\lib\site-packages\recommenders\evaluation\python_evaluation.py:1511: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to p

In [19]:
print("Precision:\t%f" % eval_precision,
      "Precision@K:\t%f" % eval_precision_k,
      "Recall:\t%f" % eval_recall,
      "Recall@K:\t%f" % eval_recall_k,
      "MAE:\t%f" % eval_mae,
      "RMSE:\t%f" % eval_rmse,
      "NDCG:\t%f" % eval_ndcg,
      "Novelty:\t%f" % eval_novelty,
      "Serendipity:\t%f" % eval_serendipity,
      "Catalog coverage:\t%f" % eval_catalog_coverage,
      "Distributional coverage:\t%f" % eval_distributional_coverage,
      sep='\n')

Precision:	0.490446
Precision@K:	0.359873
Recall:	0.029589
Recall@K:	0.183517
MAE:	1.477410
RMSE:	1.867713
NDCG:	0.490446
Novelty:	7.991052
Serendipity:	0.699576
Catalog coverage:	0.059074
Distributional coverage:	5.070852


In [20]:
eval_historical_item_novelty

,itemID,item_novelty
0,1,7.868173
1,2,9.594690
2,3,10.172235
3,4,8.845875
4,5,10.150209
...,...,...
1637,1676,16.194603
1638,1678,16.194603
1639,1679,16.194603
1640,1680,16.194603


In [21]:
eval_user_item_serendipity

,userID,itemID,user_item_serendipity
0,1,100,0.710753
1,2,300,0.731844
2,3,286,0.738688
3,4,286,0.729389
4,5,161,0.676235
...,...,...,...
938,939,294,0.746151
939,940,258,0.737345
940,941,288,0.705455
941,942,286,0.760926


In [22]:
eval_user_serendipity

,userID,user_serendipity
0,1,0.710753
1,2,0.731844
2,3,0.738688
3,4,0.729389
4,5,0.676235
...,...,...
938,939,0.746151
939,940,0.737345
940,941,0.705455
941,942,0.760926


## References

1. Rendle, S., Freudenthaler, C., Gantner, Z., & Schmidt-Thieme, L. (2009, June). BPR: Bayesian personalized ranking from implicit feedback. https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf
2. Pan, R., Zhou, Y., Cao, B., Liu, N. N., Lukose, R., Scholz, M., & Yang, Q. (2008, December). One-class collaborative filtering. https://cseweb.ucsd.edu/classes/fa17/cse291-b/reading/04781145.pdf
3. **Cornac** - A Comparative Framework for Multimodal Recommender Systems. https://cornac.preferred.ai/